# L.A. trash collection - change by commodity codes

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import datetime as dt

In [3]:
import altair as alt
import altair_latimes as lat
import matplotlib.pyplot as plt

In [4]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Read the collection data

In [5]:
src = pd.read_csv(
    "data/processed/all_data.csv",
    low_memory=False,
    parse_dates=[
        "datetime",
    ],
    dtype={
        "year": "str",
        "quarter": "str",
        "day": "str",
        "month": "str",
        "weekday": "str",
    },
)

### Exclude 2021

In [6]:
df = src[src["year"] != "2021"].copy()

In [7]:
df["month_year"] = pd.to_datetime(df["month_year"], format="%Y-%m")

In [8]:
df.commoditydescription.value_counts()

Refuse                       492833
Yard Trimmings               272348
Recycling                    238728
Regular Bulky                146132
Commercial Refuse             42857
CARE Teams                    30407
Illegal Dumping/SP            26644
White Goods                   15478
Rec. and Parks                14784
E-Waste                       10941
Washington Crew                6372
Dead Animals                   6187
Commercial Recycling           3355
Horse Manure                   2092
Commercial Yard Trimmings       786
A Bridge Home Route             703
OHS Cleanups                    656
Christmas Tree                  636
Commercial Bulky Item           324
Refuse- Grit                    185
Name: commoditydescription, dtype: int64

### Group by commodity codes

In [9]:
ccodes = (
    df.groupby(["year", "commoditycode", "commoditydescription"])
    .agg({"tons": sum})
    .sort_values(by=["year", "tons"], ascending=False)
    .reset_index()
)

In [10]:
ccodes.head(10)

,year,commoditycode,commoditydescription,tons
0,2020,A,Refuse,889267.50
1,2020,Y,Yard Trimmings,472152.98
2,2020,R,Recycling,259601.52
3,2020,B,Regular Bulky,55589.83
4,2020,CA,Commercial Refuse,35271.34
5,2020,P,Rec. and Parks,24670.58
6,2020,I,Illegal Dumping/SP,14722.56
7,2020,C,CARE Teams,6354.14
8,2020,M,Horse Manure,3888.38
9,2020,H,Washington Crew,3742.41


In [11]:
ccodes_pivot = pd.pivot_table(
    ccodes,
    columns="year",
    values="tons",
    index="commoditydescription",
    aggfunc={"tons": sum},
).reset_index()

In [12]:
before = ["2016", "2017", "2018", "2019"]

In [13]:
ccodes_pivot["before"] = ccodes_pivot[before].mean(axis=1).round(2)

In [14]:
ccodes_pivot["change_pct"] = (
    (ccodes_pivot["2020"] - ccodes_pivot["before"]) / ccodes_pivot["before"] * 100
).round(2)

In [15]:
ccodes_pivot["diff"] = ccodes_pivot["2020"] - ccodes_pivot["before"]

In [17]:
ccodes_pivot.sort_values("change_pct", ascending=False)

In [43]:
ccodes_pivot = ccodes_pivot.drop([0, 3], axis=0)

In [44]:
alt.Chart(ccodes_pivot).mark_bar().encode(
    x=alt.X(
        "diff:Q",
        title="Tons",
        axis=alt.Axis(
            format=",.0f",
            tickCount=4,
            gridColor="#dddddd",
            offset=6,
            tickSize=0,
            domainOpacity=0,
        ),
    ),
    y=alt.Y(
        "commoditydescription:O",
        title=" ",
        sort="-x",
        axis=alt.Axis(
            tickCount=4,
            gridColor="#dddddd",
            offset=6,
            tickSize=0,
            domainOpacity=0,
        ),
    ),
    color=alt.condition(
        alt.datum.change_pct > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("orange"),  # The negative color
    ),
).properties(
    width=600,
    height=500,
    title="Change in trash collection, by commodity type, 2019 vs. 2020",
).configure_view(
    strokeOpacity=0
).configure_axis(
    labelFontSize=13
)

alt.Chart(...)

In [45]:
alt.Chart(ccodes_pivot).mark_bar().encode(
    x=alt.X(
        "change_pct:Q",
        title="% change in tons",
        axis=alt.Axis(
            format=",.0f",
            tickCount=4,
            gridColor="#dddddd",
            offset=6,
            tickSize=0,
            domainOpacity=0,
        ),
    ),
    y=alt.Y(
        "commoditydescription:O",
        title=" ",
        sort="-x",
        axis=alt.Axis(
            tickCount=4,
            gridColor="#dddddd",
            offset=6,
            tickSize=0,
            domainOpacity=0,
        ),
    ),
    color=alt.condition(
        alt.datum.change_pct > 0,
        alt.value("steelblue"),  # The positive color
        alt.value("orange"),  # The negative color
    ),
).properties(
    width=600,
    height=500,
    title="Change in trash collection, by commodity type, 2019 vs. 2020",
).configure_view(
    strokeOpacity=0
).configure_axis(
    labelFontSize=13
)

alt.Chart(...)